# Polymorphism with classes

Polymorphism is supported with classes. In the code below, we have two classes, `C` and `D`, that are structurally monomorphic, but nominally polymorphic, and a function `f` which takes as input either an instance of `C` or `D`.

In [1]:
from typing import Union
import s6

class C:
  def __init__(self):
    self.x = 2

class D:
  def __init__(self):
    self.x = 3  

@s6.jit
def f(a : Union[C or D]):
  return a.x

f(D())
f(C())

2

When S6 compiles `f`, it will consider the polymorphic case, as we invoked it with instances of both classes:

In [2]:
s6.inspect(f).force_compile()
print(s6.inspect(f).strongjit)

type_feedback @2 polymorphic, either D+x#5925 or C+x#5928

function f {
&0: [ %1 ]                                                  // entry point
  bytecode_begin @0 fastlocals [%1]                         // LOAD_FAST 2275608802.py:14
  %3 = get_object_dict %1 dictoffset $16 type $0
  deoptimize_if_safepoint not %3, @2 stack [%1] fastlocals [%1] increfs [%1], "Specialized instance attribute load received an object without __dict__" // LOAD_ATTR 2275608802.py:14
  %5 = get_instance_class_id %3
  %6 = constant $5925
  %7 = cmp eq i64 %5, %6
  br %7, &9, &14

&9:                                                         // preds: &0
  %10 = deoptimized_asynchronously?
  deoptimize_if_safepoint %10, @2 stack [%1] fastlocals [%1] increfs [%1], "Assumptions were made about class behavior that were invalidated asynchronously" // LOAD_ATTR 2275608802.py:14
  %12 = load_from_dict %3, $0, split
  jmp &22 [ %12 ]

&14:                                                        // preds: &0
  %15 = co

### What do we get when `f` is compiled

We start with the `type_feedback` information at the top of the strongjit, and go over a few strongjit instructions.

```
type_feedback @2 polymorphic, either D+x#5925 or C+x#5928
```
Contains some type information about a variable (in parameter of the function, in this case). In particular,
* @2 refers to a bytecode offset within PyCodeObject::co_instructions.
* Class names look like `TYPE+attrs#NUMBER`, where `TYPE` is the type name, `attrs` is the instance attributes added on top of the type, and `NUMBER` is the unique identifier of this class in S6. For example, `D+x#5925` is `D()`, with `x` added as attribute (`class D: def __init__(self): self.x = 2`).

```
bytecode_begin @0 fastlocals [%1]    // LOAD_FAST 2275608802.py:14
```
marks the start of a bytecode boundary. This is for debugging and also for interpreting purposes -- if we ever deoptimize outside of a bytecode boundary, we can interpret the strongjit IR until we get to a boundary, and at that point it is safe to bail out to the interpreter.
* `@0`: the bytecode offset (`f_lasti`) to bail out to the interpreter with
* `fastlocals [%1]`: Defines the contents of `f_fastlocals[]`, which is the current local variable state (There’s also `stack` which defines the content of the value stack).

```
%3 = get_object_dict %1 dictoffset $16 type $0
```
Obtains the `__dict__` from `%1`. The `dictoffset` and type arguments give hints about where we expect the dict to be within `%1`. This dict extraction is being used for a type check, so if it is not there, we return `nullptr`.

```
deoptimize_if_safepoint not %3, @2 stack [%1] fastlocals [%1] increfs [%1], "Specialized instance attribute load received an object without __dict__" // LOAD_ATTR 2275608802.py:14
```
If `%3 == nullptr`, S6 bails out to the interpreter. The rest of the line gives all the information needed to bail out successfully, just like bytecode_begin.

```
%5 = get_instance_class_id %3
```
Obtains the class ID of an object, from its `__dict__`. `%3` is the `__dict__` (as obtained by `get_object_dict`).

```
%10 = deoptimized_asynchronously?
```
Checks the code object's _did something outrageous happen?_ flag, i.e., if some assumption has been broken. For example, if a type modification happened (`class X: pass; X.x = 42`) and we relied on that type for some optimization, then we invalidate all generated code that relied on it. We will set a _deoptimize now_ flag on all code objects affected, and this instruction checks it before we do something that is no longer correct.

```
%12 = load_from_dict %3, $0, split
```
Is a _fast load_ of an object's field. Given a `__dict__` in `%3`, load the zero-th item (`$0`) from it. The dict is known to be a split dict (`ma_keys != ma_values`) so all this needs to do is load `((PyDictObject*)%3).ma_values[0]`.

```
incref notnull %23
```
Increments the reference count of `%23`, which is known not to be null.

```
advance_profile_counter $3
```
The Oracle maintains a profile counter which is initialized to some large value and counts down over time. When it hits zero or goes below, we trigger a _maybe optimize this code?_ event. The instruction `advance_profile_counter` modifies this counter by however many bytecode instructions’ worth of jitted code we’ve just executed.

```
%4 = unbox long %1
```
Attempts to unbox `PyObject %1` as an integer. If `%1` isn’t of type `PyLong`, or is outside the range of a 64-bit signed integer, it fails (setting the overflowed flag).

```
%5 = overflowed? %4
```
Returns true if the previous instruction overflowed or failed (applies to any integer operation or unbox operation)

```
%11 = box long %8
```
Boxes up the integer value in `%8` into a new `PyLong`.
